In [1]:
import pandas as pd
import numpy as np

#read in data from csv from folder ../data
crime_df = pd.read_csv('../data/cf_apprissAppend_va2_10_17_2022.csv')
medical_df = pd.read_csv('../data/CF_medicalData_v2.csv')
PII_df = pd.read_csv('../data/dev_sample_HeartTable_10_17_2022.csv')

C:\Users\cayde\AppData\Local\Temp\ipykernel_1860\739831198.py:5: DtypeWarning: Columns (26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  crime_df = pd.read_csv('../data/cf_apprissAppend_va2_10_17_2022.csv')
C:\Users\cayde\AppData\Local\Temp\ipykernel_1860\739831198.py:6: DtypeWarning: Columns (16,17,18,19,20,21,22,23,34,36,49,50,51,52,53,54,55,56,57,58,59,60,62,63,64,74,75,76,78,79,80,85,87,88,89,90,91,92,93,97,98,99,108,123,124) have mixed types. Specify dtype option on import or set low_memory=False.
  medical_df = pd.read_csv('../data/CF_medicalData_v2.csv')


In [2]:
# cut down the dataframe medical_df to only the columns we need 'source_id', 'ENDate'
medical_df = medical_df[['source_id', 'ENDate', 'MiddleName']]
# join the dataframes PII_df and medical_df left on source_id
PII_med_df = pd.merge(PII_df, medical_df, on='source_id', how='left')
PII_med_df = PII_med_df[['source_id', 'FIRST_NAME', 'MiddleName', 'LAST_NAME', 'MASTER_DOB', 'ENDate']]



In [3]:
# cut down crime_df to only the columns we need 'source_id', 'SenderId' 'PersonFirstName' 'PersonMiddleName' 'PersonLastName' 'BookingDate'
crime_df = crime_df[['PK','source_id', 'SenderId', 'PersonFirstName', 'PersonMiddleName', 'PersonLastName', 'BookingDate', 'PersonYearOfBirth']]
# rename the column "pk" to booking_event_id
crime_df = crime_df.rename(columns={'PK': 'booking_event_id'})


In [4]:
# join the dataframes PII_med_df and crime_df right on source_id
Crime_PII_df = pd.merge(PII_med_df, crime_df, on='source_id', how='right')

In [5]:
# for the columns PersonFirstName, PersonMiddleName, PersonLastName remove the first and last character
Crime_PII_df['PersonFirstName'] = Crime_PII_df['PersonFirstName'].str[1:-1]
Crime_PII_df['PersonMiddleName'] = Crime_PII_df['PersonMiddleName'].str[1:-1]
Crime_PII_df['PersonLastName'] = Crime_PII_df['PersonLastName'].str[1:-1]
Crime_PII_df['PersonYearOfBirth'] = Crime_PII_df['PersonYearOfBirth'].str[1:-1]
# this will get rid of all of the [] in the columns

In [6]:
#import regular expressions
import re
# # check to see whether each source_id is associated with a unique sender_id
# Crime_PII_df.groupby('source_id').SenderId.nunique().value_counts()

# For all rows where ENDate is not null, check whether BookingDate occurs after ENDate
Crime_PII_df['DiedBeforeBookingCharge'] = pd.to_datetime(Crime_PII_df['BookingDate']) > pd.to_datetime(Crime_PII_df['ENDate'])

def YOB_match(row):
    if type(row['PersonYearOfBirth']) == str:
        pass
    else:
        return False
    # use regular expression to find if 'FIRST_NAME' column in Crime_PII_df contains 'PersonFirstName' column in Crime_PII_df
    if re.search(row['PersonYearOfBirth'], row['MASTER_DOB']):
        return True
    else:
        return False

Crime_PII_df['YOB_match'] = Crime_PII_df.apply(YOB_match, axis=1)

In [7]:
#import regular expressions
import re

# cast all strings to all lower case
Crime_PII_df['FIRST_NAME'] = Crime_PII_df['FIRST_NAME'].str.lower()
Crime_PII_df['PersonFirstName'] = Crime_PII_df['PersonFirstName'].str.lower()
Crime_PII_df['LAST_NAME'] = Crime_PII_df['LAST_NAME'].str.lower()
Crime_PII_df['PersonLastName'] = Crime_PII_df['PersonLastName'].str.lower()
Crime_PII_df['PersonMiddleName'] = Crime_PII_df['PersonMiddleName'].str.lower()
Crime_PII_df['MiddleName'] = Crime_PII_df['MiddleName'].str.lower()

# create a function that returns a boolean value for whether the first name in the crime data matches the first name in the PII data
def first_name_match(row):
    if type(row['PersonFirstName']) == str:
        pass
    else:
        return False
    # use regular expression to find if 'FIRST_NAME' column in Crime_PII_df contains 'PersonFirstName' column in Crime_PII_df
    if re.search(row['FIRST_NAME'], row['PersonFirstName']):
        return True
    else:
        return False

# create a function that returns a boolean value for whether the last name in the crime data matches the last name in the PII data
def last_name_match(row):
    if type(row['PersonLastName']) == str:
        pass
    else:
        return False
    # use regular expression to find if 'LAST_NAME' column in Crime_PII_df contains 'PersonLastName' column in Crime_PII_df
    if re.search(row['LAST_NAME'], row['PersonLastName']):
        return True
    else:
        return False

# create a function that returns a boolean value for whether the middle name in the crime data matches the middle name in the PII data
def middle_name_match(row):
    # convert middle_name to a string
    middle_name = str(row['MiddleName'])
    # if middle_name is null, return False
    if middle_name == 'nan':
        return 'no_middle_name_on_medical_record'
    # if row['PersonMiddleName'] is an empty string, return False
    if row['PersonMiddleName'] == '':
        return 'no_middle_name_on_appriss_record'
    # check if row['PersonMiddleName'] can be searched with re.search without throwing an error
    if type(row['PersonMiddleName']) == str:
        pass
    else:
        return 'False'
    # use regular expression to find if 'PersonMiddleName' column in Crime_PII_df contains 'PersonMiddleName' column in Crime_PII_df
    if re.search(middle_name, row['PersonMiddleName']):
        return 'True'
    else:
        return 'False'

Crime_PII_df['FN_match'] = Crime_PII_df.apply(first_name_match, axis=1)
Crime_PII_df['LN_match'] = Crime_PII_df.apply(last_name_match, axis=1)
Crime_PII_df['MN_match'] = Crime_PII_df.apply(middle_name_match, axis=1)

In [8]:
# different combinations of first, middle, and last name matches
Crime_PII_df['FN_LN_match'] = Crime_PII_df['FN_match'] & Crime_PII_df['LN_match']
Crime_PII_df['FN_MN_match'] = Crime_PII_df['FN_match'] & Crime_PII_df['MN_match']
Crime_PII_df['LN_MN_match'] = Crime_PII_df['LN_match'] & Crime_PII_df['MN_match']
Crime_PII_df['FN_LN_MN_match'] = Crime_PII_df['FN_match'] & Crime_PII_df['LN_match'] & Crime_PII_df['MN_match']

In [9]:
# turn PersonLastName into a list
Crime_PII_df['PersonLastName_list'] = Crime_PII_df['PersonLastName'].str.split(',')

# print person last name list for row 33
Crime_PII_df['PersonLastName_list'][33]

['AARON BROWN', ' BROWN']

In [10]:

def has_multi_names(row):
    if len(row['PersonLastName_list']) > 1:
        return True
    else:
        return False

def count_multi_names(row):
    if len(row['PersonLastName_list']) >= 1:
    #write a switch statment that checks to see if the list of names is between 1-5, 6-10, 11-20, 20+
        if len(row['PersonLastName_list']) <= 5:
            return '1-5'
        elif len(row['PersonLastName_list']) <= 10:
            return '6-10'
        elif len(row['PersonLastName_list']) <= 20:
            return '11-20'
        else:
            return '20+'

Crime_PII_df['has_mult_names'] = Crime_PII_df.apply(has_multi_names, axis=1)
Crime_PII_df['count_mult_names'] = Crime_PII_df.apply(count_multi_names, axis=1)


In [11]:
# write dataframe to csv
Crime_PII_df.to_csv('../data/out/Confidence_match_crimedata.csv')

In [15]:
import pandas as pd
import numpy as np

# write dataframe to xlsx
Crime_PII_df.to_excel('../data/out/Confidence_match_crimedata.xlsx', sheet_name='Sheet1')